# 05 - Model Comparison
Train all three models and compare metrics/plots in one place.


In [1]:
from pathlib import Path
import sys

ROOT = Path("..").resolve()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import numpy as np
import pandas as pd


In [2]:
from pathlib import Path

import pandas as pd
import joblib

from src.eval import compute_full_metrics
from src.plots import plot_model_comparison
from _common import load_dataset, prepare_features, ROOT
from src.split import SplitConfig

# MLflow
MLFLOW_ENABLED = True
MLFLOW_EXPERIMENT = "f1-laptime"
MLFLOW_TRACKING_URI = (ROOT / "mlruns").as_uri()
MLFLOW_RUN_NAME = "model_comparison"

# Verify MLflow availability
if MLFLOW_ENABLED:
    try:
        import mlflow  # noqa: F401
        print(f"MLflow available: {mlflow.__version__}")
    except Exception:
        print("MLflow not installed; set MLFLOW_ENABLED=False or install mlflow.")
        MLFLOW_ENABLED = False

# Model loading (comparison uses saved models only)
MODELS_DIR = ROOT / "reports" / "models"
MODEL_PATHS = {
    "Linear": MODELS_DIR / "linear.joblib",
    "XGBoost": MODELS_DIR / "xgboost.joblib",
    "Deep MLP": MODELS_DIR / "deep_mlp.joblib",
}

missing = [name for name, path in MODEL_PATHS.items() if not path.exists()]
if missing:
    raise FileNotFoundError(f"Missing saved models: {missing}. Expected in {MODELS_DIR}")

models = {name: joblib.load(path) for name, path in MODEL_PATHS.items()}

# Full 2025 evaluation split
split_config = SplitConfig(test_rounds=None)
df, metadata = load_dataset()
_, _, _, test_df, features = prepare_features(df, metadata, split_config=split_config)

X_test = test_df[features]
y_test = test_df["LapTimeSeconds"].to_numpy()

metrics_rows = []
preds = {}
for name, model in models.items():
    preds[name] = model.predict(X_test)
    scores = compute_full_metrics(y_test, preds[name], n_features=len(features))
    scores["model"] = name
    metrics_rows.append(scores)

metrics = pd.DataFrame(metrics_rows).sort_values("mae").reset_index(drop=True)


MLflow available: 3.8.1


In [3]:
plot_model_comparison(metrics)


In [4]:
# MLflow logging
if MLFLOW_ENABLED:
    try:
        import mlflow
    except ImportError:
        print("MLflow not installed; skipping MLflow logging.")
    else:
        def _coerce_params(params):
            return {k: str(v) for k, v in params.items()}

        if MLFLOW_TRACKING_URI:
            mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        mlflow.set_experiment(MLFLOW_EXPERIMENT)
        with mlflow.start_run(run_name=MLFLOW_RUN_NAME or "model_comparison"):
            mlflow.log_param("comparison_source", "saved_models")
            mlflow.log_param("test_rounds", split_config.test_rounds)

            for model_name, estimator in models.items():
                with mlflow.start_run(run_name=model_name, nested=True):
                    if hasattr(estimator, "best_params_"):
                        mlflow.log_params(_coerce_params(estimator.best_params_))
                    elif hasattr(estimator, "get_params"):
                        params = {k: v for k, v in estimator.get_params().items() if k.startswith("model__")}
                        mlflow.log_params(_coerce_params(params))

                    row = metrics[metrics["model"] == model_name].iloc[0]
                    for metric in ("mae", "rmse", "r2", "mape_pct", "smape_pct"):
                        if metric in row:
                            mlflow.log_metric("test_" + metric, float(row[metric]))

                    if MODEL_PATHS[model_name].exists():
                        mlflow.log_artifact(str(MODEL_PATHS[model_name]), artifact_path="models")


c:\Users\tvcar\Desktop\FOM\2. Semester\Maschinelles Lernen\ml_f1\.venv\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning:

The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.

